In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

Способы ввода данных:
1. Ввод с клавиатуры
2. Случайная генерация
3. Импорт файла csv

In [ ]:
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import numpy as np
import scipy, csv

In [ ]:
def transpose_mtrx(a): # Транспонирование матрицы
    a = np.array(a)
    trns = a.transpose()
    return trns


def input_matrix(x,y): # Функция создания матрицы вручную
    def input_values():  
        try:
            num[i][j] = int(input('Введите элемент строки {} столбца {}: '.format(i+1, j+1)))
            return num[i][j]
        except ValueError:
            print('Ошибка ввода!\nВведите элемент заново:\n')
            return input_values()
    
    try:
        num = [[0 for n in range(y)] for nn in range(x)]
        print('Введите матрицу весовых коэффициентов поэлементно:')
        for i in range(x):
            for j in range(y):
                input_values()
        return num
    
    except ValueError:
        print('\nОшибка!\nВведите заново количество сток/столбцов:\n')
        return input_matrix()


def PesOpVal(num, c): # Функция просчитывания стратегии по критериям оптимизма/пессимизма/Вальда
    min_and_max = []
    for i in range(len(num)):
        row = []
        row.append(min(num[i]))
        row.append(max(num[i]))
        min_and_max.append(row)

    Vald = []
    for i in range(len(num)):
        Vald.append(min(num[i]))
    for i in range(len(min_and_max)):
        min_and_max[i].append(Vald[i])
    
    answr = []
    transpose_min_and_max = transpose_mtrx(min_and_max)
    answr.append(min(transpose_min_and_max[0]))
    answr.append(max(transpose_min_and_max[1]))
    answr.append(max(transpose_min_and_max[2]))
    
    # Пессимизм
    pes = []
    for i in range(len(num)):
        if min(num[i]) == answr[0]:
            pes.append(c[i])
    print('\nОптимальная стратегия игрока А по критерию пессимизма: {}'.format(pes[0]))
    print('Цена игры оптимальной стратегии по критерию пессимизма: {}\n'.format(answr[0]))
    
    # Оптимизм
    opt = []
    for i in range(len(num)):
        if max(num[i]) == answr[1]:
            opt.append(c[i])
    print('Оптимальная стратегия игрока А по критерию оптимизма: {}'.format(opt[0]))
    print('Цена игры оптимальной стратегии по критерию оптимизма: {}\n'.format(answr[1]))
    
    # Вальд
    strategy_Vald = []
    for i in range(len(num)):
        if Vald[i] == answr[2]:
            strategy_Vald.append(c[i])
    print('Оптимальная стратегия игрока А по критерию Вальда: {}'.format(strategy_Vald[0]))
    print('Цена игры оптимальной стратегии по критерию Вальда: {}\n'.format(answr[2]))
    return answr


def Gurvits(num, c): # Линейная свертка склонности к риску по критерию Гурвица + Гистограмма максимумов 
    min_and_max = []
    for i in range(len(num)):
        row = []
        row.append(min(num[i]))
        row.append(max(num[i]))
        min_and_max.append(row)
    
    matrix_Gurvits = []
    for i in range(len(min_and_max)):
        row = []
        for j in range(11):
            row.append((j/10) * min_and_max[i][1] + (1 - j/10) * min_and_max[i][0])
        matrix_Gurvits.append(row)
    transpose_matrix_Gurvits = transpose_mtrx(matrix_Gurvits)
    
    answr = [max(transpose_matrix_Gurvits[i]) for i in range(len(transpose_matrix_Gurvits))]
    
    DF_matrix_Gurvits = list(matrix_Gurvits)
    DF_matrix_Gurvits.append(answr)
    DF_index = list(c)
    DF_index.append('max')
    print('Линейная свертка склонности к риску по критерию Гурвица:\n', pd.DataFrame(DF_matrix_Gurvits, index = DF_index, columns=[(i/10) for i in range(11)]))
    
    # Формирование графика
    fig, ax = plt.subplots(figsize = (15,8))
    x = np.arange(0, 11)
    for i in range(len(matrix_Gurvits)):
        ax.bar(x - (len(matrix_Gurvits)/10) + 0.1 + i/5, matrix_Gurvits[i], width = 0.2, label = f'{c[i]}')
    
    plt.plot(x, answr, label = 'max', c = 'm')
    plt.xlabel('Величина склонности к риску')
    plt.ylabel('Цена игры')
    plt.title('Гистограмма максимумов линейной свертки по критерию Гурвица')
    plt.legend()
    plt.show()


def Sevige(num, c):  # Оптимальная стратегия по критерию Сэвиджа 
    transpose_num = transpose_mtrx(num)
    
    max_values = [max(transpose_num[i]) for i in range(len(transpose_num))]
    
    matrix_Sevige = []
    for i in range(len(num)):
        row = []
        for j in range(len(max_values)):
            row.append(max_values[j] - num[i][j])
        matrix_Sevige.append(row)
        
    answr = min([max(matrix_Sevige[i]) for i in range(len(matrix_Sevige))])

    strategy_Sevige = []
    for i in range(len(matrix_Sevige)):
        if max(matrix_Sevige[i]) == answr:
            strategy_Sevige.append(c[i])
    
    print('Оптимальная стратегия игрока А по критерию Сэвиджа: {}'.format(strategy_Sevige[0]))
    print('Величина минимальной недополученной прибыли по критерию Сэвиджа: {}\n'.format(answr))


def sum_strategy(): # Защита от неверного ввода данных
    try:
        x = int(input('Введите количество стратегий для компании А: '))
        if x <= 0:
            print('Количество стратегий должно быть больше 0!')
            return sum_strategy()
        y = int(input('Введите количество состояний природы П: '))
        if y <= 0:
            print('Количество стратегий должно быть больше 0!')
            return sum_strategy()
        return [x,y]
    except ValueError:
        print('Количество стратегий должно быть числом!')
        return sum_strategy()


# -------------------------------------------------

way = int(input('Выберите способ ввода данных: '))

# Ввод с клавиатуры
if way == 1:
    sum_strategy = sum_strategy()
    sum_strategy_company, sum_strategy_nature = sum_strategy[0], sum_strategy[1]
    
    name_strategy_company = []
    for i in range(sum_strategy_company):
        name = input('Введите название {} стратегии компании А: '.format(i+1))
        name_strategy_company.append('A{} - {}'.format(i+1, name))
    
    name_strategy_nature = []
    for i in range(sum_strategy_nature):
        name = input('Введите название {} состояния природы П: '.format(i+1))
        name_strategy_nature.append('A{} - {}'.format(i+1, name))
    
    num = input_matrix(sum_strategy_company, sum_strategy_nature)
    
    PesOpVal(num, name_strategy_company)
    Gurvits(num, name_strategy_company)
    Sevige(num, name_strategy_company)

# Случайная генерация
elif way == 2:
    sum_strategy = sum_strategy()
    sum_strategy_company, sum_strategy_nature = sum_strategy[0], sum_strategy[1]
    
    name_strategy_company, name_strategy_nature = [], []
    for i in range(sum_strategy_company):
        name_strategy_company.append('A' + str(i+1))
        name_strategy_nature.append('П' + str(i+1))
    
    print('Введите диапазон значений для матрицы весовых коэффициентов :')
    min_value = int(input('Введите минимальное значение: ')) 
    max_value = int(input('Введите максимальное значение: '))
    
    num = [[randint(min_value, max_value) for i in range(sum_strategy_nature)] for j in range(sum_strategy_company)]
    print('\nВесовая матрица:\n', pd.DataFrame(num, index = name_strategy_company, columns = name_strategy_nature), '\n')
    
    PesOpVal(num, name_strategy_company)
    Gurvits(num, name_strategy_company)
    Sevige(num, name_strategy_company)

# Импорт csv файла
elif way == 3:
    num = []
    try:
        file_way = input('Введите путь к файлу: ')
        with open(file_way, 'r', newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=';')
            for row in spamreader:
                num.append(row)

        # Для образования списка с названиями колонок матрицы
        name_strategy_nature = num[0]
        name_strategy_nature = name_strategy_nature[1:]
        del num[0]

        # Для образования списка с названиями строк
        name_strategy_company = []
        for num_value, value in enumerate(num):
            name_strategy_company.append(value[0])
            del num[num_value][0]
        
        # Перевод коэффициентов из str в тип int
        for num_row, row in enumerate(num):
            for num_value, value in enumerate(row):
                num[num_row][num_value] = int(value)

        print('\nВесовая матрица:\n', pd.DataFrame(num, index = name_strategy_company, columns = name_strategy_nature), '\n')

        PesOpVal(num, name_strategy_company)
        Gurvits(num, name_strategy_company)
        Sevige(num, name_strategy_company)
    
    except:
        print('Файл не найден. Попробуйте еще раз')
    
else:
    print('Такого способа ввода данных нет!')